# Основные точки лица

[карта точек](images/mesh_map.jpg)

In [1]:
#pip install opencv-python mediapipe msvc-runtime
import mediapipe as mp
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

Тут реализовано нахождение всего мэша лица + центра глаза (а вот контроль за зрачком будет далее):

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_face_mesh=mp.solutions.face_mesh


drawing_spec_points = mp_drawing.DrawingSpec(   # точки 
          color=(0,0,255),
          thickness=1,
          circle_radius=2
        )

drawing_spec_lines = mp_drawing.DrawingSpec(  # линии соединения
          color=(255,0,0),
          thickness=1,
          circle_radius=1
        )

cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                           min_tracking_confidence=0.5,
                           max_num_faces=1,
                           refine_landmarks=False) as face_mesh:
    while True:
        ret,image=cap.read()
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image.flags.writeable=False
        results=face_mesh.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                
                # Пример контроля левого глаза
                landmark_down = face_landmarks.landmark[374]
                landmark_up = face_landmarks.landmark[386]

                x, y = int((landmark_up.x+ landmark_down.x)/2  * image.shape[1]), int((landmark_up.y+ landmark_down.y)/2 * image.shape[0])

                # Display the first landmark point in red
                cv2.circle(image, (x, y), 5, (255, 255, 0), -1)

                # Пример контроля правого глаза
                landmark_down = face_landmarks.landmark[159]
                landmark_up = face_landmarks.landmark[145]

                x, y = int((landmark_up.x+ landmark_down.x)/2  * image.shape[1]), int((landmark_up.y+ landmark_down.y)/2 * image.shape[0])

                # Display the first landmark point in red
                cv2.circle(image, (x, y), 5, (0, 255, 255), -1)

                mp_drawing.draw_landmarks(image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=drawing_spec_points,
                        connection_drawing_spec=drawing_spec_lines)

        cv2.imshow("Face Mesh", image)
        k=cv2.waitKey(1)
        if k==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


---

## Контроль за зрачком:

In [2]:
# refine_landmarks = True -> включает детекцию зрачков

# координаты точек мэша
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
LEFT_EYE = [263, 249, 390, 373, 374, 380, 381, 382, 362, 398, 384, 385, 386, 387, 388, 466]

LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

In [3]:
mp_drawing=mp.solutions.drawing_utils
mp_face_mesh=mp.solutions.face_mesh


drawing_spec_points = mp_drawing.DrawingSpec(   # точки 
          color=(0,0,255),
          thickness=1,
          circle_radius=2
        )

drawing_spec_lines = mp_drawing.DrawingSpec(  # линии соединения
          color=(255,0,0),
          thickness=1,
          circle_radius=1
        )

cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                           min_tracking_confidence=0.5,
                           max_num_faces=1,
                           refine_landmarks=True) as face_mesh:
    while True:
        ret,image=cap.read()
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image.flags.writeable=False
        results=face_mesh.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mesh_points = np.array([[int(p.x * image.shape[1]), int(p.y * image.shape[0])] for p in face_landmarks.landmark], dtype=int)

                # обводка глаз
                cv2.polylines(image, [mesh_points[RIGHT_EYE]], True, (0,255,0), 1, cv2.LINE_AA)
                cv2.polylines(image, [mesh_points[LEFT_EYE]], True, (0,255,200), 1, cv2.LINE_AA)

                # обводка зрачков (всего 4 точки так что получился ромб)
                #cv2.polylines(image, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv2.LINE_AA)
                #cv2.polylines(image, [mesh_points[RIGHT_IRIS]], True, (0,255,200), 1, cv2.LINE_AA)

                # For the right iris
                (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                center_right = (int(r_cx), int(r_cy))
                radius_right = int(r_radius)
                cv2.circle(image, center_right, radius_right, (0, 100, 255), 1)  # контур зрачка
                cv2.circle(image, center_right, 1, (0, 100, 255), 2)  # центр

                # For the left iris
                (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
                center_left = (int(l_cx), int(l_cy))
                radius_left = int(l_radius)
                cv2.circle(image, center_left, radius_left, (0, 100, 255), 1)  # контур зрачка
                cv2.circle(image, center_left, 1, (0, 100, 255), 2)  # центр
                

        cv2.imshow("Face Mesh", image)
        k=cv2.waitKey(1)
        if k==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
